In [ ]:
from yugiquery import *

header("Speed Duel")

---

Table of Contents
=================

*   [1  Data aquisition](#Data-aquisition)
    *   [1.1  Fetch online data](#Fetch-online-data)
    *   [1.2  Merge data](#Merge-data)
*   [2  Check changes](#Check-changes)
    *   [2.1  Load previous data](#Load-previous-data)
    *   [2.2  Generate changelog](#Generate-changelog)
    *   [2.3  Save data](#Save-data)
*   [3  Data visualization](#Data-visualization)
    *   [3.1  Skill cards](#Skill-cards)
        *   [3.1.1  Character](#Character)
        *   [3.1.2  Property](#Property)
    *   [3.2  Property](#Property)
    *   [3.3  Archseries](#Archseries)
    *   [3.4  TCG Speed Duel status](#TCG-Speed-Duel-status)
        *   *   [3.4.0.1  By card type](#By-card-type)
            *   [3.4.0.2  By monster type](#By-monster-type)
            *   [3.4.0.3  By archseries](#By-archseries)
        *   [3.4.1  TCG Speed Duel vs. TCG status](#TCG-Speed-Duel-vs.-TCG-status)
*   [4  Epilogue](#Epilogue)
    *   [4.1  HTML export](#HTML-export)
<!--     *   [4.2  Git](#Git) -->

# Data aquisition

## Fetch online data

In [ ]:
# Timestamp
timestamp = pd.Timestamp.now()

In [ ]:
# Fetch skill cards
skill_df = fetch_skill()

# Fetch deck cards
speed_df = fetch_speed()

## Merge data

In [ ]:
full_df = (
    pd.concat([speed_df, skill_df], ignore_index=True, axis=0)
    .sort_values("Name")
    .reset_index(drop=True)
)
print("Data merged")

# Check changes

## Load previous data

In [ ]:
## Get list of files
files_list = sorted(
    glob.glob("../data/all_speed_*.csv"), key=os.path.getctime, reverse=True
)
# Get second newest file if exist
if files_list:
    latest_file = files_list[0]
    # Load csv avoiding converting "NA" to NaN
    previous_df = pd.read_csv(
        latest_file, dtype=object, keep_default_na=False, na_values=[""]
    )
    # Correct tuples
    tuple_cols = ["Secondary type", "Effect type", "Archseries", "Artwork"]
    for col in tuple_cols:
        previous_df[col] = previous_df[col].dropna().apply(literal_eval)
    # Force dtypes to match current df
    previous_df = previous_df.astype(
        full_df[previous_df.columns.intersection(full_df.columns)].dtypes.to_dict()
    )
    previous_ts = pd.to_datetime(
        os.path.basename(latest_file).split("_")[-1].split(".csv")[0]
    )
    print("File loaded")
else:
    previous_df = None
    print("No older files")

## Generate changelog

In [ ]:
if previous_df is None:
    changelog = None
    print("Skipped")
else:
    changelog = generate_changelog(previous_df, full_df, col="Name")
    if not changelog.empty:
        display(changelog)
        changelog.to_csv(
            f'../data/speed_changelog_{previous_ts.isoformat(timespec="minutes")}_{timestamp.isoformat(timespec="minutes")}.bz2',
            index=True,
        )
        print("Changelog saved")

## Save data

In [ ]:
if changelog is not None and changelog.empty:
    print("No changes. New data not saved")
else:
    full_df.to_csv(
        f'../data/all_speed_{timestamp.isoformat(timespec="minutes")}.bz2', index=False
    )
    print("Data saved")

# Data visualization

In [ ]:
full_df

## Skill cards

### Character

In [ ]:
print(
    "Total number of characters portrayed in skill cards:",
    skill_df["Character"].nunique(),
)

In [ ]:
skill_df.drop("Card type", axis=1).sort_values("Character")

In [ ]:
skill_color = colors_dict["Skill Card"]
skill_df["Character"].value_counts().plot.bar(
    figsize=(18, 6), grid=True, rot=90, color=skill_color, title="Character"
)
plt.show()

### Property

In [ ]:
print("Total number of properties:", skill_df["Property"].nunique())

In [ ]:
skill_df.groupby("Property").nunique()

In [ ]:
skill_color = colors_dict["Skill Card"]
skill_df["Property"].value_counts().plot.bar(
    figsize=(18, 6), grid=True, rot=0, color=skill_color, title="Property"
)
plt.show()

## Property

In [ ]:
print("Total number of properties:", full_df["Property"].nunique())

In [ ]:
full_df.groupby("Property").nunique()

In [ ]:
property_unstack = full_df.groupby("Property")["Card type"].value_counts().unstack()
skill_st_colors = [colors_dict[i] for i in property_unstack.columns]
property_unstack.plot.bar(
    figsize=(18, 6), stacked=True, grid=True, rot=45, color=skill_st_colors
)
plt.show()

## Archseries

In [ ]:
exploded_archseries = full_df.explode("Archseries")
print("Total number of Archseries:", exploded_archseries["Archseries"].nunique())

In [ ]:
exploded_archseries.groupby("Archseries").nunique()

In [ ]:
exploded_archseries["Archseries"].value_counts().plot.barh(
    figsize=(10, 50), grid=True, title="Archtypes/Series"
)
plt.show()

## TCG Speed Duel status

In [ ]:
print(
    "Total number of TCG Speed Duel status:", full_df["TCG Speed Duel status"].nunique()
)

In [ ]:
full_df.groupby("TCG Speed Duel status", dropna=False).nunique()

In [ ]:
full_df["TCG Speed Duel status"].value_counts(dropna=False).plot.bar(
    figsize=(18, 6), logy=True, grid=True, rot=45, title="TCG status"
)
plt.show()

#### By card type

In [ ]:
# Remove unlimited
tcg_speed_crosstab = pd.crosstab(full_df["Card type"], full_df["TCG Speed Duel status"])
tcg_speed_crosstab

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(
    tcg_speed_crosstab[tcg_speed_crosstab > 0],
    annot=True,
    fmt="g",
    cmap="viridis",
    norm=mc.LogNorm(),
)
plt.show()

#### By monster type

In [ ]:
# Remove unlimited
tcg_speed_crosstab_b = pd.crosstab(
    full_df["Monster type"], full_df["TCG Speed Duel status"]
)
tcg_speed_crosstab_b

In [ ]:
plt.figure(figsize=(20, 5))
sns.heatmap(
    tcg_speed_crosstab_b[tcg_speed_crosstab_b > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
)
plt.show()

#### By archseries

In [ ]:
# Remove unlimited
tcg_crosstab_c = pd.crosstab(
    exploded_archseries["Archseries"],
    exploded_archseries["TCG Speed Duel status"],
    margins=True,
)
tcg_crosstab_c

### TCG Speed Duel vs. TCG status

In [ ]:
cg_crosstab = pd.crosstab(
    full_df["TCG status"], full_df["TCG Speed Duel status"], dropna=False, margins=False
)
cg_crosstab

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(
    cg_crosstab[cg_crosstab > 0],
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=mc.LogNorm(),
)
plt.show()

# Epilogue

In [ ]:
benchmark("speed", timestamp)

In [ ]:
footer()

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
! jupyter nbconvert Speed.ipynb --output-dir='../' --to=HTML --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='exclude' --TemplateExporter.exclude_input=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True

## Git

In [ ]:
! git add "../*[Ss]peed*"

In [ ]:
! git commit -m {"'Speed duel update-" + timestamp.isoformat() + "'"}